<a href="https://colab.research.google.com/github/njanrepo/MLEProject/blob/main/task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor


In [2]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 7.5 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=e3031929274ba09dd539adb2158de70bd976d2ed65b8927dce60acf65c6dd898
  Stored in directory: /root/.cache/pip/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
import yfinance as yf

#Get data


In [4]:
def get_nyse_ticker_list():
  others_list = 'ftp://ftp.nasdaqtrader.com/symboldirectory/otherlisted.txt'
  ticker_df = pd.read_csv(others_list, sep='|')
  company_nyse = ticker_df[ticker_df['Exchange'] == 'N']
  return company_nyse['ACT Symbol'].values.tolist()

s=get_nyse_ticker_list()
len(s)

3642

In [5]:
def get_sp500_list():
  payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
  first_table = payload[0]
  sp500df = first_table
  sp500_list = sp500df.Symbol.values.tolist()
  return sp500_list



#Calculate Market Capitalization


In [28]:
ticker='JPM'
def calc_market_capitalization(symbol):
  symbol_data = yf.download(symbol, period='1d')
  print(symbol_data)
  return round((symbol_data['Volume'][0] * symbol_data['Close'][0]),2)
print(calc_market_capitalization(ticker))

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close   Volume
Date                                                                           
2021-11-11  167.520004  168.679993  166.899994  167.610001  167.610001  5811400
974048757.55


In [ ]:
sp_list = get_sp500_list()
s_info = []
i =0
for t in sp_list:
  print(t)
  i +=1
  print(i)
  inn = yf.Ticker(t).info
  s_info.append(inn)

In [47]:
sp_df = pd.DataFrame(s_info)

In [74]:
df=yf.download(tickers='JPM', period='180d')

[*********************100%***********************]  1 of 1 completed


In [85]:
def ask_user(stocks = None):
  if(stocks != None):
    return stocks;
  how_many_stocks = input('How many stocks you want to analyse? \n Enter atleast 10 and max 500 \n')
  correct = False
  while not correct:
    try:
      how_many_stocks = int(how_many_stocks)
      if(how_many_stocks > 500 or how_many_stocks < 10):
        raise ValueError
      correct = True
    except:
      print("Please enter correct number 10 to 500")
      how_many_stocks = input('How many stocks you want to analyse? \n Enter atleast 10 and max 500 \n')
  return how_many_stocks    

In [86]:
def make_X_and_Y(close_prices, period=6):
  X = []
  y = []
  for i in range(len(close_prices)- period):
    X.append(close_prices[i:i+period])
    y.append(close_prices[i+period])
  return X,y

In [82]:
def return_model(X, y):
  clf = RandomForestRegressor()
  clf.fit(X,y)
  return clf

In [114]:
def predict_price(symbol, period=6):
  print("Analysing :: ", symbol )
  data = yf.download(symbol, period='6mo')
  price = data.Close
  features, dep_var = make_X_and_Y(list(price))
  features = np.array(features)
  dep_var = np.array(dep_var)
  clf=return_model(features, dep_var)
  latest_feature = np.array(list(price)[-period:]).reshape((1,-1))
  s_dict['symbol'] = symbol
  s_dict['volatility'] = round(price.std(), 2)
  s_dict['tomorrow'] = round(clf.predict(latest_feature)[0], 2)
  s_dict['today'] = round(price[-1],2)
  return s_dict

In [117]:
predict_price('TSLA')

Analysing ::  TSLA
[*********************100%***********************]  1 of 1 completed
                  Open         High  ...    Adj Close    Volume
Date                                 ...                       
2021-11-11  1102.77002  1104.969971  ...  1063.510010  22176100
2021-11-12  1047.50000  1054.389893  ...  1022.190125  16530163

[2 rows x 6 columns]


{'symbol': 'TSLA', 'today': 1022.19, 'tomorrow': 1058.86, 'volatility': 154.18}

In [100]:


list_of_symbols = get_sp500_list()
symbols_not_found = ['BRK.B', 'BF.B']
period = 6
stk_data = []
how_many_stocks = ask_user(12)
for sym in list_of_symbols[0:how_many_stocks]:
  if sym not in symbols_not_found:
    s_dict = predict_price(sym)
    print(s_dict)

Analysing ::  MMM
[*********************100%***********************]  1 of 1 completed
{'symbol': 'MMM', 'volatility': 9.49, 'tomorrow': 180.39, 'today': 183.35}
Analysing ::  ABT
[*********************100%***********************]  1 of 1 completed
{'symbol': 'ABT', 'volatility': 5.94, 'tomorrow': 128.33, 'today': 127.91}
Analysing ::  ABBV
[*********************100%***********************]  1 of 1 completed
{'symbol': 'ABBV', 'volatility': 3.96, 'tomorrow': 117.2, 'today': 116.91}
Analysing ::  ABMD
[*********************100%***********************]  1 of 1 completed
{'symbol': 'ABMD', 'volatility': 27.27, 'tomorrow': 361.97, 'today': 363.3}
Analysing ::  ACN
[*********************100%***********************]  1 of 1 completed
{'symbol': 'ACN', 'volatility': 25.8, 'tomorrow': 367.08, 'today': 370.46}
Analysing ::  ATVI
[*********************100%***********************]  1 of 1 completed
{'symbol': 'ATVI', 'volatility': 8.67, 'tomorrow': 68.06, 'today': 69.5}
Analysing ::  ADBE
[******

In [54]:
def predict_price():
  

12